In [3]:

import torch
import torch.nn as nn
import torch.nn.functional as F

import math
import warnings
import math
import einops
import torch.utils.checkpoint
import torch.nn.functional as F
def _no_grad_trunc_normal_(tensor, mean, std, a, b):
    # Cut & paste from PyTorch official master until it's in a few official releases - RW
    # Method based on https://people.sc.fsu.edu/~jburkardt/presentations/truncated_normal.pdf
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    if (mean < a - 2 * std) or (mean > b + 2 * std):
        warnings.warn("mean is more than 2 std from [a, b] in nn.init.trunc_normal_. "
                      "The distribution of values may be incorrect.",
                      stacklevel=2)

    with torch.no_grad():
        # Values are generated by using a truncated uniform distribution and
        # then using the inverse CDF for the normal distribution.
        # Get upper and lower cdf values
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)

        # Uniformly fill tensor with values from [l, u], then translate to
        # [2l-1, 2u-1].
        tensor.uniform_(2 * l - 1, 2 * u - 1)

        # Use inverse cdf transform for normal distribution to get truncated
        # standard normal
        tensor.erfinv_()

        # Transform to proper mean, std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)

        # Clamp to ensure it's in the proper range
        tensor.clamp_(min=a, max=b)
        return tensor


def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    # type: (Tensor, float, float, float, float) -> Tensor
    r"""Fills the input Tensor with values drawn from a truncated
    normal distribution. The values are effectively drawn from the
    normal distribution :math:`\mathcal{N}(\text{mean}, \text{std}^2)`
    with values outside :math:`[a, b]` redrawn until they are within
    the bounds. The method used for generating the random values works
    best when :math:`a \leq \text{mean} \leq b`.
    Args:
        tensor: an n-dimensional `torch.Tensor`
        mean: the mean of the normal distribution
        std: the standard deviation of the normal distribution
        a: the minimum cutoff value
        b: the maximum cutoff value
    Examples:
        >>> w = torch.empty(3, 5)
        >>> nn.init.trunc_normal_(w)
    """
    return _no_grad_trunc_normal_(tensor, mean, std, a, b)


def drop_path(x, drop_prob: float = 0., training: bool = False):
    """Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).

    This is the same as the DropConnect impl I created for EfficientNet, etc networks, however,
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ... I've opted for
    changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use
    'survival rate' as the argument.

    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output


class DropPath(nn.Module):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)


class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x
if hasattr(torch.nn.functional, 'scaled_dot_product_attention'):
    ATTENTION_MODE = 'flash'
else:
    try:
        import xformers
        import xformers.ops
        ATTENTION_MODE = 'xformers'
    except:
        ATTENTION_MODE = 'math'
print(f'attention mode is {ATTENTION_MODE}')


def timestep_embedding(timesteps, dim, max_period=10000):
    """
    Create sinusoidal timestep embeddings.

    :param timesteps: a 1-D Tensor of N indices, one per batch element.
                      These may be fractional.
    :param dim: the dimension of the output.
    :param max_period: controls the minimum frequency of the embeddings.
    :return: an [N x dim] Tensor of positional embeddings.
    """
    half = dim // 2
    freqs = torch.exp(
        -math.log(max_period) * torch.arange(start=0, end=half, dtype=torch.float32) / half
    ).to(device=timesteps.device)
    args = timesteps[:, None].float() * freqs[None]
    embedding = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
    if dim % 2:
        embedding = torch.cat([embedding, torch.zeros_like(embedding[:, :1])], dim=-1)
    return embedding


def patchify(imgs, patch_size):
    x = einops.rearrange(imgs, 'B C (h p1) (w p2) -> B (h w) (p1 p2 C)', p1=patch_size, p2=patch_size)
    return x


def unpatchify(x, channels=128):
    patch_size = int((x.shape[2] // channels) ** 0.5)
    h = w = int(x.shape[1] ** .5)
    assert h * w == x.shape[1] and patch_size ** 2 * channels == x.shape[2]
    x = einops.rearrange(x, 'B (h w) (p1 p2 C) -> B C (h p1) (w p2)', h=h, p1=patch_size, p2=patch_size)
    return x


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, L, C = x.shape

        qkv = self.qkv(x)
#         print("qkv", qkv.shape)
        if ATTENTION_MODE == 'flash':
            qkv = einops.rearrange(qkv, 'B L (K H D) -> K B H L D', K=3, H=self.num_heads).float()
            q, k, v = qkv[0], qkv[1], qkv[2]  # B H L D
            x = torch.nn.functional.scaled_dot_product_attention(q, k, v)
            x = einops.rearrange(x, 'B H L D -> B L (H D)')
        elif ATTENTION_MODE == 'xformers':
            qkv = einops.rearrange(qkv, 'B L (K H D) -> K B L H D', K=3, H=self.num_heads)
            q, k, v = qkv[0], qkv[1], qkv[2]  # B L H D
            x = xformers.ops.memory_efficient_attention(q, k, v)
            x = einops.rearrange(x, 'B L H D -> B L (H D)', H=self.num_heads)
        elif ATTENTION_MODE == 'math':
            qkv = einops.rearrange(qkv, 'B L (K H D) -> K B H L D', K=3, H=self.num_heads)
            q, k, v = qkv[0], qkv[1], qkv[2]  # B H L D
            attn = (q @ k.transpose(-2, -1)) * self.scale
            attn = attn.softmax(dim=-1)
            attn = self.attn_drop(attn)
            x = (attn @ v).transpose(1, 2).reshape(B, L, C)
        else:
            raise NotImplemented

        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class Block(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm, skip=False, use_checkpoint=False):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale)
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer)
        self.skip_linear = nn.Linear(2 * dim, dim) if skip else None
        self.use_checkpoint = use_checkpoint

    def forward(self, x, skip=None):
        if self.use_checkpoint:
            return torch.utils.checkpoint.checkpoint(self._forward, x, skip)
        else:
            return self._forward(x, skip)

    def _forward(self, x, skip=None):
        if self.skip_linear is not None:
            x = self.skip_linear(torch.cat([x, skip], dim=-1))
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x


class PatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """
    def __init__(self, patch_size, in_chans=3, embed_dim=768):
        super().__init__()
        self.patch_size = patch_size
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        assert H % self.patch_size == 0 and W % self.patch_size == 0
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x


class UViT(nn.Module):
    def __init__(self, img_size=224, patch_size=8, in_chans=3, embed_dim=2048, depth=8, num_heads=8, mlp_ratio=4.,
                 qkv_bias=True, qk_scale=None, norm_layer=nn.LayerNorm, mlp_time_embed=False, num_classes=None,
                 use_checkpoint=False, conv=True, skip=True):
        super().__init__()
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        self.num_classes = num_classes
        self.in_chans = in_chans

        self.patch_embed = PatchEmbed(patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)
        num_patches = (img_size // patch_size) ** 2

        self.time_embed = nn.Sequential(
            nn.Linear(embed_dim, 4 * embed_dim),
            nn.SiLU(),
            nn.Linear(4 * embed_dim, embed_dim),
        ) if mlp_time_embed else nn.Identity()

        if self.num_classes is not None:
            self.label_emb = nn.Embedding(self.num_classes, embed_dim)
            self.extras = 2
        else:
            self.extras = 1

        self.pos_embed = nn.Parameter(torch.zeros(1, self.extras + num_patches, embed_dim))

        self.in_blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                norm_layer=norm_layer, use_checkpoint=use_checkpoint)
            for _ in range(depth // 2)])

        self.mid_block = nn.Sequential(
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                norm_layer=norm_layer, use_checkpoint=use_checkpoint)
        )
#         ,
#             SSPCAB(channels = num_patches+1))
#         self.SDB = SSPCAB(channels=197)
        self.out_blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                norm_layer=norm_layer, skip=skip, use_checkpoint=use_checkpoint)
            for _ in range(depth // 2)])

        self.norm = norm_layer(embed_dim)
        self.patch_dim = patch_size ** 2 * in_chans
        self.decoder_pred = nn.Linear(embed_dim, self.patch_dim, bias=True)
        self.final_layer = nn.Conv2d(self.in_chans, self.in_chans, 3, padding=1) if conv else nn.Identity()

        trunc_normal_(self.pos_embed, std=.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'pos_embed'}
    
class GroupNorm32(nn.GroupNorm):
    def forward(self, x):
        return super().forward(x.float()).type(x.dtype)


def zero_module(module):
    """
    Zero out the parameters of a module and return it.
    """
    for p in module.parameters():
        p.detach().zero_()
    return module


def update_ema_params(target, source, decay_rate=0.9999):
    targParams = dict(target.named_parameters())
    srcParams = dict(source.named_parameters())
    for k in targParams:
        targParams[k].data.mul_(decay_rate).add_(srcParams[k].data, alpha=1 - decay_rate)


class EncoderWithSkipConnections(nn.Module):
    def __init__(self, in_chans, cnn_channels):
        super(EncoderWithSkipConnections, self).__init__()

        # Define the CNN layers for feature extraction
        self.conv1 = nn.Conv2d(in_chans, cnn_channels, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(cnn_channels, cnn_channels * 2, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        # Save feature maps at specific layers for skip connections
        skip1 = self.pool1(self.relu1(self.conv1(x)))
        skip2 = self.pool2(self.relu2(self.conv2(skip1)))
        x = skip2
        return x, [skip1, skip2]  # Return feature maps at different scales (skip connections)



class DecoderWithSkipConnections(nn.Module):
    def __init__(self, cnn_channels, image_size):
        super(DecoderWithSkipConnections, self).__init__()
        
        # Calculate the size of the feature map after the CNN layers and patch embedding
        cnn_out_size = image_size // 4  # Assuming two max-pooling layers with kernel_size=2 and stride=2
        patch_embed_size = cnn_channels * 2  # Output channels of the patch embedding
        
        # Transposed convolutional layers to upsample the feature map
        self.transconv1 = nn.ConvTranspose2d(patch_embed_size, cnn_channels, kernel_size=4, stride=2, padding=1)
        self.transconv2 = nn.ConvTranspose2d(cnn_channels * 2, 3, kernel_size=4, stride=2, padding=1)
        
    def forward(self, x, skip_connection):
        
#         x = torch.cat([x, skip_connection[0]], dim=1)  # Concatenate along the channel dimension
        # Apply transposed convolutional layers to upsample the feature map
        x = F.relu(self.transconv1(x))
        
        # Combine with skip connection from the encoder
        x = torch.cat([x, skip_connection[0]], dim=1)  # Concatenate along the channel dimension
        
        x = torch.sigmoid(self.transconv2(x))  # Use sigmoid activation to restrict pixel values to [0, 1]
        return x


class CUViT(UViT):
    def __init__(self, img_size=224, in_chans=3, cnn_channels=64, patch_size=4, embed_dim=2048, depth=12,
                 num_heads=8, mlp_ratio=4., qkv_bias=True, qk_scale=None, norm_layer=nn.LayerNorm,
                 mlp_time_embed=False, num_classes=None, use_checkpoint=False, conv=True, skip=True):
        super().__init__()
        self.num_features = self.embed_dim = embed_dim
        self.num_classes = num_classes
        self.in_chans = in_chans

        self.cnn = EncoderWithSkipConnections(in_chans, cnn_channels)

        # Calculate the number of patches after CNN layers
        cnn_out_size = img_size // 4  # Assuming two max-pooling layers with kernel_size=2 and stride=2
        num_patches = (cnn_out_size // patch_size) ** 2
        patch_embed_size = cnn_channels * 2  # Output channels of the patch embedding
        # Patch Embedding layer
        self.patch_embed = nn.Conv2d(cnn_channels * 2, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.pos_embed = nn.Parameter(torch.zeros(1, self.extras + num_patches, embed_dim))
        self.in_blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                norm_layer=norm_layer, use_checkpoint=use_checkpoint)
            for _ in range(depth // 2)])

        self.mid_block = nn.Sequential(
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                norm_layer=norm_layer, use_checkpoint=use_checkpoint)
        )
#         ,
#             SSPCAB(channels = num_patches+1))
#         self.SDB = SSPCAB(channels=197)
        self.out_blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                norm_layer=norm_layer, skip=skip, use_checkpoint=use_checkpoint)
            for _ in range(depth // 2)])

        self.norm = norm_layer(embed_dim)
        self.patch_dim = patch_size ** 2 * patch_embed_size
        self.decoder_pred = nn.Linear(embed_dim, self.patch_dim, bias=True)
        self.final_layer = nn.Conv2d(patch_embed_size, patch_embed_size, 3, padding=1) if conv else nn.Identity()
        self.decoder_with_skip = DecoderWithSkipConnections(cnn_channels, img_size)
        trunc_normal_(self.pos_embed, std=.02)
        self.apply(self._init_weights)

    def forward(self, x, timesteps, y=None):
#         print(timesteps)
        x, cskips = self.cnn(x)
        B1, C1, h1, w1  = x.shape
#         print(x.shape)
        x = self.patch_embed(x)
#         print(x.shape)
        x = x.flatten(2).transpose(1, 2)
#         print(x.shape)
        B, L, D = x.shape
#         print(x)
        time_token = self.time_embed(timestep_embedding(timesteps, self.embed_dim))
        time_token = time_token.unsqueeze(dim=1)
        x = torch.cat((time_token, x), dim=1)
        if self.num_classes is not None:
            label_emb = self.label_emb(y)
            label_emb = label_emb.unsqueeze(dim=1)
            x = torch.cat((label_emb, x), dim=1)
#             print("Yess")
#         print(self.pos_embed.shape)
        x = x + self.pos_embed
#         print(self.pos_embed.shape)
        skips = []
        for blk in self.in_blocks:
#             print(x.shape)
            x = blk(x)
#             print(x.shape)
            skips.append(x)
#         bf = x
        x = self.mid_block(x)
#         at = x
        for blk in self.out_blocks:
            x = blk(x, skips.pop())
        x = self.norm(x)
        print(x.shape)
        x = self.decoder_pred(x)
        print(x.shape)
        assert x.size(1) == self.extras + L
        print(x.shape)
        x = x[:, self.extras:, :]
        print(x.shape)
        x = unpatchify(x, C1)
        
        x = self.final_layer(x)
        print(x.shape)
        x = self.decoder_with_skip(x, cskips)
        return x, 0, 0
#         return x, 0, 0


attention mode is flash


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EncoderWithSkipConnections(nn.Module):
    def __init__(self, in_chans, cnn_channels):
        super(EncoderWithSkipConnections, self).__init__()

        # Define the CNN layers for feature extraction
        self.conv1 = nn.Conv2d(in_chans, cnn_channels, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(cnn_channels, cnn_channels * 2, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        # Save feature maps at specific layers for skip connections
        skip1 = self.pool1(self.relu1(self.conv1(x)))
        skip2 = self.pool2(self.relu2(self.conv2(skip1)))
        x = skip2
        return x, [skip1, skip2]  # Return feature maps at different scales (skip connections)



class DecoderWithSkipConnections(nn.Module):
    def __init__(self, cnn_channels, image_size, out_channel = 3):
        super(DecoderWithSkipConnections, self).__init__()
        
        # Calculate the size of the feature map after the CNN layers and patch embedding
        cnn_out_size = image_size // 4  # Assuming two max-pooling layers with kernel_size=2 and stride=2
        patch_embed_size = cnn_channels * 2  # Output channels of the patch embedding
        
        # Transposed convolutional layers to upsample the feature map
        self.transconv1 = nn.ConvTranspose2d(patch_embed_size, cnn_channels, kernel_size=4, stride=2, padding=1)
        self.transconv2 = nn.ConvTranspose2d(cnn_channels * 2, out_channel, kernel_size=4, stride=2, padding=1)
        
    def forward(self, x, skip_connection):
        
#         x = torch.cat([x, skip_connection[0]], dim=1)  # Concatenate along the channel dimension
        # Apply transposed convolutional layers to upsample the feature map
        x = F.relu(self.transconv1(x))
        
        # Combine with skip connection from the encoder
        x = torch.cat([x, skip_connection[0]], dim=1)  # Concatenate along the channel dimension
        
        x = torch.sigmoid(self.transconv2(x))  # Use sigmoid activation to restrict pixel values to [0, 1]
        return x


class CUViT(UViT):
    def __init__(self, img_size=224, in_chans=3, cnn_channels=64, patch_size=4, embed_dim=2048, depth=12,
                 num_heads=8, mlp_ratio=4., qkv_bias=True, qk_scale=None, norm_layer=nn.LayerNorm,
                 mlp_time_embed=False, num_classes=None, use_checkpoint=False, conv=True, skip=True):
        super().__init__()
        self.num_features = self.embed_dim = embed_dim
        self.num_classes = num_classes
        self.in_chans = in_chans

        self.cnn = EncoderWithSkipConnections(in_chans, cnn_channels)

        # Calculate the number of patches after CNN layers
        cnn_out_size = img_size // 4  # Assuming two max-pooling layers with kernel_size=2 and stride=2
        num_patches = (cnn_out_size // patch_size) ** 2
        patch_embed_size = cnn_channels * 2  # Output channels of the patch embedding
        # Patch Embedding layer
        self.patch_embed = nn.Conv2d(cnn_channels * 2, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.pos_embed = nn.Parameter(torch.zeros(1, self.extras + num_patches, embed_dim))
        self.in_blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                norm_layer=norm_layer, use_checkpoint=use_checkpoint)
            for _ in range(depth // 2)])

        self.mid_block = nn.Sequential(
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                norm_layer=norm_layer, use_checkpoint=use_checkpoint)
        )
#         ,
#             SSPCAB(channels = num_patches+1))
#         self.SDB = SSPCAB(channels=197)
        self.out_blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                norm_layer=norm_layer, skip=skip, use_checkpoint=use_checkpoint)
            for _ in range(depth // 2)])

        self.norm = norm_layer(embed_dim)
        self.patch_dim = patch_size ** 2 * patch_embed_size
        self.decoder_pred = nn.Linear(embed_dim, self.patch_dim, bias=True)
        self.final_layer = nn.Conv2d(patch_embed_size, patch_embed_size, 3, padding=1) if conv else nn.Identity()
        self.decoder_with_skip = DecoderWithSkipConnections(cnn_channels, img_size, out_channel=in_chans)
        trunc_normal_(self.pos_embed, std=.02)
        self.apply(self._init_weights)

    def forward(self, x, timesteps, y=None):
#         print(timesteps)
        x, cskips = self.cnn(x)
        B1, C1, h1, w1  = x.shape
#         print(x.shape)
        x = self.patch_embed(x)
#         print(x.shape)
        x = x.flatten(2).transpose(1, 2)
#         print(x.shape)
        B, L, D = x.shape
#         print(x)
        time_token = self.time_embed(timestep_embedding(timesteps, self.embed_dim))
        time_token = time_token.unsqueeze(dim=1)
        x = torch.cat((time_token, x), dim=1)
        if self.num_classes is not None:
            label_emb = self.label_emb(y)
            label_emb = label_emb.unsqueeze(dim=1)
            x = torch.cat((label_emb, x), dim=1)
#             print("Yess")
#         print(self.pos_embed.shape)
        x = x + self.pos_embed
#         print(self.pos_embed.shape)
        skips = []
        for blk in self.in_blocks:
#             print(x.shape)
            x = blk(x)
#             print(x.shape)
            skips.append(x)
#         bf = x
        x = self.mid_block(x)
#         at = x
        for blk in self.out_blocks:
            x = blk(x, skips.pop())
        x = self.norm(x)
        print(x.shape)
        x = self.decoder_pred(x)
        print(x.shape)
        assert x.size(1) == self.extras + L
        print(x.shape)
        x = x[:, self.extras:, :]
        print(x.shape)
        x = unpatchify(x, C1)
        
        x = self.final_layer(x)
        print(x.shape)
        x = self.decoder_with_skip(x, cskips)
        return x
#         return x, 0, 0

# Instantiate the modified UViT model with CNN layers before patch embedding
model = CUViT(in_chans=1).cuda()

# Print the model architecture



In [5]:
x_test = torch.rand(1, 1, 224, 224).cuda()

In [6]:
t_batch = torch.randint(1, 1000, (x_test.shape[0],)).cuda()

In [8]:
y_test = model(x_test, t_batch)

torch.Size([1, 197, 2048])
torch.Size([1, 197, 2048])
torch.Size([1, 197, 2048])
torch.Size([1, 196, 2048])
torch.Size([1, 128, 56, 56])


In [8]:
# model.patch_dim

In [9]:
y_test

tensor([[[[0.2127, 0.6206, 0.3043,  ..., 0.5703, 0.2700, 0.4566],
          [0.5011, 0.4393, 0.5418,  ..., 0.4319, 0.5179, 0.3167],
          [0.3486, 0.5738, 0.2678,  ..., 0.5313, 0.3125, 0.4544],
          ...,
          [0.4966, 0.3781, 0.4762,  ..., 0.4475, 0.3625, 0.3640],
          [0.3715, 0.6531, 0.2985,  ..., 0.5218, 0.2599, 0.5546],
          [0.4169, 0.2317, 0.4574,  ..., 0.2889, 0.4674, 0.2882]]]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)

In [1]:
import torch

In [10]:
torch.cuda.is_available()

True